In [1]:
import numpy as np;
import pandas as pd;

In [2]:
ROOT_DIR = "../"
mu = 0.01

```
                P1
               /. \
     ----------    -------
     |                   |
     10                 10
     |                   |
     P1.1 ---- 10 ----- P1.2           
     / \                / \
    10  10            10  10 
   /      \          /      \
  P1.1.1  P1.1.2  P1.2.1.  P1.2.2 
```

In [3]:
initDataDF = pd.read_csv(ROOT_DIR + "/data/sample_init_data.csv")

distanceData = pd.read_csv(ROOT_DIR + "/data/sample_distance_matrix.csv")

r = len(initDataDF.index)

In [4]:
distanceMatrix = distanceData['value'].to_numpy().reshape(r,r)
distanceMatrix

array([[ 0, 10, 10, 20, 20, 20, 20],
       [10,  0, 10, 10, 10, 20, 20],
       [10, 10,  0, 20, 20, 10, 10],
       [20, 10, 20,  0, 20, 30, 30],
       [20, 10, 20, 20,  0, 30, 30],
       [20, 20, 10, 30, 30,  0, 20],
       [20, 20, 10, 30, 30, 20,  0]])

In [10]:
def getSij(i, j, distanceMatrix, initDataDF):
    return(initDataDF[distanceMatrix[i] <= distanceMatrix[j]]['S'].sum())

In [6]:
getSij(3,1,distanceMatrix, initDataDF)

7500

I'm not sure about the getSij function. It looks like you are taking all the regions which are closer to i than j. What we need is all the regions closer to i than d(i,j). Think of a circle centered at Ri of radius d(R1,R2) and include all the regions in that circle. 

In [7]:
def getTij(i,j, distanceMatrix, initDataDF):
    Ni = initDataDF.iloc[i]['S']
    Ti = mu*Ni
    if(i==j):
        return Ni - Ti;
    
    Nj = initDataDF.iloc[j]['S']
    Sij = getSij(i,j,distanceMatrix, initDataDF)
    Tij = Ti * ((Ni * Nj) /((Ni + Sij)*(Ni + Nj + Sij)))
    return Tij
    
    

In [8]:
resultsDF = pd.DataFrame(columns=['iName', 'jName', 'Tij'], );
for i in range(0,r-1):
    for j in range(0,r-1):
        Tij = getTij(i,j,distanceMatrix, initDataDF)
        resultsDF = resultsDF.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij},ignore_index=True)
 

In [9]:
resultsDF.to_csv(ROOT_DIR + "/output/tmatrix_results.csv")